### 1. Введение

#### 1.1. Организационная информация


Основная коммуникация: Telegram `DL - СПбГУ - 2023`. Если вас там нет, напишите `@snikolenko`, чтобы он вас туда добавил.

Материалы и домашние работы: `emkn`, GitHub: https://github.com/norsage/dl-mkn.git

Аттестация: по результатам результатов решений нескольких прикладных задач, где потребуется побить бейзлайн

Упражнения после занятия: опциональны, но очень полезны для развития интуиции


#### 1.2. План на сегодня: знакомство с PyTorch

1. Тензоры и операции над ними
2. Граф вычислений и автоматическое дифференцирование
3. Реализация новых операций с помощью `torch.autograd.Function`

#### 1.3. Подготовка окружения

1) Локально с помощью conda
   
   (можно установить по инструкции https://docs.conda.io/projects/miniconda/en/latest/index.html#quick-command-line-install)
   ```bash
   # создаём окружение dl-course
   conda create -n dl-course -y python=3.10
   # активируем окружение
   conda activate dl-course
   # устанавливаем Jupyter
   conda install jupyter notebook -c conda-forge

   # Устанавливаем pytorch (https://pytorch.org/get-started/locally/)
   # Пример для Linux / Windows с поддержкой GPU:
   conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
   # Для MacOS с ускорением MPS:
   # conda install pytorch::pytorch torchvision torchaudio -c pytorch
   ```

   Запустить ноутбук можно как в браузере:
   
   `jupyter notebook 01_pytorch_intro.ipynb`

   так и в VSCode, просто открыв файл и выбрав кернел для исполнения в правом верхнем углу (потребуется поставить расширения `Python` и `Jupyter`)
2) Используем Сolab https://colab.research.google.com/

### 2. PyTorch basics

In [1]:
import torch
print(torch.__version__)
print(f"CUDA available: {torch.cuda.is_available()}")

2.0.1
CUDA available: False


#### 2.1. Тензоры, способы создания и атрибуты

https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html

Создать тензор можно многими способами:
1. Напрямую из объектов
2. Из массивов `numpy`
3. Из других тензоров
4. С константными и случайными значениями
5. Используя специальные функции для особых случаев

Основные атрибуты: ранг (`dim`), размерности (`shape`), тип значений (`type`), место размещения (`device`)

In [2]:
# из списка
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
# посмотрим сам тензор и его атрибуты
print(x)
print(
    "\nRank: ", x.dim(),
    "\nShape: ", x.shape,
    "\nDevice: ", x.device,
    "\ntype: ", x.type(),
)

tensor([[1, 2, 3],
        [4, 5, 6]])

Rank:  2 
Shape:  torch.Size([2, 3]) 
Device:  cpu 
type:  torch.LongTensor


In [ ]:
import numpy as np
# создание из numpy
arr = np.random.normal(size=(3, 2))
x = torch.from_numpy(arr)  # shape = (3, 2)

# из других тензоров, объединяя несколько в один
y = x + 1
z = torch.cat([x, y], dim=0)  # shape = (6, 2)
z = torch.cat([x, y], dim=1)  # shape = (3, 4)

# константы и специальные случаи
x = torch.ones(size=(4,), dtype=bool)
x = torch.eye(4, dtype=int)
x = torch.linspace(start=0, end=5, steps=11)

# случайные величины
x = torch.rand(size=(4,))  # U[0, 1]
x = torch.randn(size=(2, 2))  # N(0, 1)


# если нужно зафиксировать seed
torch.manual_seed(4)
x = torch.bernoulli(
    torch.full((3, 2), 0.2)
)
x


#### 2.2. Простые операции, функции, линейная алгебра

https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

Различных операций сотни, можно выделить несколько групп по назначению:
- создание (`linspace`, `arange`, `full`)
- индексация, срезы, объединения (`argwhere`, `cat`, `stack`, `tile`, `gather`)
- генерация случайных величин (`normal`, `bernoulli`, `randperm`)
- сериализация / десериализация (`save`, `load`)
- математические операции (`log`, `deg2rad`, `clamp`, `sigmoid`)
- агрегирование (`sum`, `mean`, `argmax`, `quantile`)
- линейная алгебра (`torch.linalg.*`)
- FFT (`torch.fft.*`)
- обработка сигналов (`torch.signal.*`)
- нейронные сети (`torch.nn.*`)
- ...

Многие операции также реализованы как методы класса `Tensor`

In [ ]:
x = torch.arange(12).view((-1, 3, 2))  # shape: (2, 3, 2)
# срезы
x[0, 1:3, :2]  # shape: (2, 2)
# берём всё из ведущих размерностей, из последней - только второй элемент
x[:, :, 1]  # shape: (2, 3)
# лень преречислять ведущие размерности - используем эллипсис
x[..., 1]  # shape: (2, 3) - то же самое, что выше!
# получение по списку индексов

indices = torch.tensor([0, 1, 0])  # наш список индексов
print("Исходный тензор:")
print(x)
print("Получен по индексам:")

# получим срез по нашим индексам, применённым к различным размерностям:
x.take_along_dim(indices.view(-1,  1,  1), dim=0)  # shape: (3, 3, 2)
x.take_along_dim(indices.view( 1, -1,  1), dim=1)  # shape: (2, 3, 2)
x.take_along_dim(indices.view( 1,  1, -1), dim=2)  # shape: (2, 3, 3)
#x.take_along_dim(indices.view(-1, 1, 1), dim=2)  # shape: (3, 3, 2)
#x

#### 2.3. Broadcasting

https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html#in-brief-tensor-broadcasting

Некоторые операции поддерживают `broadcast`, то есть размерности аргументов автоматически расширяются до нужного размера без копирования данных

Общие правила, когда это работает:
1. Все тензоры не пустые
2. При сравнении размеров тензоров, начиная с последней:
   1. Размерности совпадают, или
   2. Одна из размерностей равна $1$, или
   3. Размерность отсутствует в одном из тензоров


Благодаря `broadcast` многие вещи получается описать лаконично.

Возможно ли сложить тензоры `x` и `y` в примерах снизу?

In [ ]:
# x=torch.empty(5,7,3)
# y=torch.empty(5,7,3)

# x=torch.empty((0,))
# y=torch.empty(2,2)

# x=torch.empty(5,3,4,1)
# y=torch.empty(  3,1,1)

# x=torch.empty(5,2,4,1)
# y=torch.empty(  3,1,1)

### 3. Autograd в PyTorch

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

#### 3.1. Вычислительный граф и дифференцирование

`autograd` строит DAG (directed acyclic graph) из объектов `torch.autograd.Function`, листья - входные тензоры, корни - выходные тензоры. Проход по графу позволяет рассчитать градиенты по правилу производной сложной функции (chain rule).

Прямой проход:
- расчёт значения выходного тензора
- построение графа и сохранение нужных для обратного прохода данных для каждой операции

Обратный проход (вызов `.backward()` у корня графа):
- расчёт градиентов и их накопление в артибуте `.grad` каждого тензора
- распространение вычислений далее до листьев графа

Рассмотрим выражение $f(x, y) = x^2 + xy + (x + y)^2$ и построим его граф:

<img src="expression_graph.png" style="background:white" width="300"/>

Запишем выражение для $f(x, y)$, задав начальные условия $x = 2.0, y = 2.0$.

In [3]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
a = x ** 2
b = x*y
c = x + y
d = a + b
e = c**2
f = d + e
f

tensor(24., grad_fn=<AddBackward0>)

`grad_fn` означает, что `f` не просто отдельный тензор, а связан с вычислительным графом и соответствует операции `Add`

Запустим backprop и убедимся, что градиенты рассчитаны правильно:

$\frac{\partial f}{\partial x} = 2x + y + 2(x + y)$

$\frac{\partial f}{\partial y} = x + 2(x + y)$

In [4]:
f.backward()
print(x.grad)
print(y.grad)

tensor(14.)
tensor(10.)


О чём стоит помнить:
- Повторная попытка выполнить `backward()` приводит к ошибке, т.к. после первого вызова граф уничтожается для экономии ресурсов. Если мы хотим сохранить граф для повторного обратного прохода, делаем `f.backward(retain_graph=True)`
- Градиенты в узлах графа не обнуляются автоматически, и при следующем вызове `backward` новое значение прибавится к старому
- Если по какой-то причине мы не хотим считать градиенты, вычисление выражения можно обернуть в контекст ```with torch.no_grad():```

#### 3.2. Отключение расчёта градиентов

https://pytorch.org/docs/stable/notes/autograd.html#locally-disable-grad-doc

Несколько способов:
1. Изменить значение атрибута тензора `requires_grad` напрямую

In [5]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
print(y.requires_grad)  # True
x.requires_grad = False
y = x + 1
print(y.requires_grad)  # False 

True
False


2. Использовать `torch.no_grad()` (как менеджер контекста или как декоратор)

In [6]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
print(x.requires_grad)  # True

# локально отключаем трекинг градиентов
with torch.no_grad():
    y = x + 1
print(y.requires_grad)  # False

True
False


In [7]:
# декоратор
def add_one(t: torch.Tensor) -> torch.Tensor:
    return t + 1

@torch.no_grad()
def add_two(t: torch.Tensor) -> torch.Tensor:
    return t + 1

x = torch.tensor(1.0, requires_grad=True)
y = add_one(x)
print(y.requires_grad)  # True
z = add_two(x)
print(z.requires_grad)  # False

True
False


3. Получить копию тензора с помощью метода `.detach()`

In [8]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
print(x.requires_grad)  # True
z = y.detach()
print(z.requires_grad)  # False

True
False


#### 3.3. Пример: логистическая регрессия

$\hat{y} = \sigma(w^T x + b)$

$\sigma(t) = \frac{1}{1 + \exp(-t)}$

$\text{CE}(y, \hat{y}) = -y \cdot \log \hat{y} - (1 - y) \log (1 - \hat{y})$

In [9]:
torch.manual_seed(42)
x = torch.ones(5)  # входной тензор
y = torch.zeros(3)  # выходной тензор
w = torch.randn(5, 3, requires_grad=True)  # параметр, хотим обновлять градиентным спуском
b = torch.randn(3, requires_grad=True)  # параметр, хотим обновлять градиентным спуском
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)  # функция потерь, хотим минимизировать
loss

tensor(2.2493, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [10]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x138989510>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x13898a3e0>


<img src="https://pytorch.org/tutorials/_images/comp-graph.png" style="background:white" width="700"/>

Напишем цикл для поиска параметров, минимизирующих функцию ошибки

Некоторые распространённые ошибки:

1. Обновление параметра на месте вне контекста `torch.no_grad()` вызовет ошибку. PyTorch такое явно запрещает

In [11]:
w = torch.tensor(1.0, requires_grad=True)
#x = torch.tensor(1.0)
f = w + 1
f.backward()
w -= w.grad

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

2. Обновление параметра через присваивание приводит к тому, что параметр больше не является листом графа, в графе образовался цикл, и следующая итерация приведёт к ошибке:

In [12]:
w = torch.tensor(1.0, requires_grad=True)
#x = torch.tensor(1.0)
f = w + 1
# первая итерация - всё как будто ок
f.backward()
w = w - w.grad  # на этом моменте `w` - больше не лист графа
f = w + 1

# вторая итерация - оказывается, что не ок
f.backward()
w = w - w.grad  # здесь w.grad is None!


/var/folders/z3/n11m8hqx40s1trg9fk01s2qh0000gp/T/ipykernel_21890/1172668910.py:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343686130/work/build/aten/src/ATen/core/TensorBody.h:491.)
  w = w - w.grad  # здесь w.grad is None!


TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

3. Внутри контекста `torch.no_grad()` параметр обновляем не на месте, а через переназначение, после этого он более не ожидает градиентов, всё ломается при вызове `.backward()`

In [13]:
w = torch.tensor(1.0, requires_grad=True)
f = w + 1
# первая итерация
f.backward()
with torch.no_grad():
    w = w - w.grad  # упс, для w теперь requires_grad = False!
f = w + 1

# вторая итерация
f.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

#### 3.4. Класс `torch.autograd.Function`

Когда может пригодиться:
1. Добавление недифференцируемых вычислений
2. Добавление операций, реализованный вне PyTorch (NumPy, SciPy, etc.)
3. Более эффективное использование ресурсов (комбинирование операций, обёртка над реализацией на `C++`)

In [14]:
class MulConstant(torch.autograd.Function):
    @staticmethod
    def forward(tensor, constant):
        return tensor * constant

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        tensor, constant = inputs
        ctx.constant = constant

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        return grad_output * ctx.constant, None

In [15]:
class Mul(torch.autograd.Function):
    @staticmethod
    def forward(tensor1, tensor2):
        return tensor1 * tensor2

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        tensor1, tensor2 = inputs
        ctx.tensor1 = tensor1
        ctx.tensor2 = tensor2

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        return grad_output * ctx.tensor2, grad_output * ctx.tensor1

In [16]:
x = torch.tensor(2.0, requires_grad=True)
mul_const = MulConstant.apply
res = mul_const(x, 4)
print("Result: ", res)
res.backward()
print("Gradient for x: ", x.grad)

Result:  tensor(8., grad_fn=<MulConstantBackward>)
Gradient for x:  tensor(4.)


In [17]:
mul = Mul.apply
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

res = mul(x, y)
print("Result: ", res)
res.backward()
print("Gradient for x: ", x.grad)
print("Gradient for y: ", y.grad)



Result:  tensor(6., grad_fn=<MulBackward>)
Gradient for x:  tensor(3.)
Gradient for y:  tensor(2.)


### 4. Упражнения

#### 4.1. Функция Power
Используя сложение и умножение, реализуйте возведение в целочисленную степень FloatTensor как функцию autograd (т.е. наследника `torch.autograd.Function`)

In [ ]:
from typing import Any

class Power(torch.autograd.Function):
    @staticmethod
    def forward(tensor, p):
        ...

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        ...

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        ...

In [ ]:
assert(torch.all(Power.apply(torch.tensor([1, 2, 3]), 0) == torch.tensor([1, 1, 1])))
assert(torch.all(Power.apply(torch.tensor([1, 2, 3]), 2) == torch.tensor([1, 4, 9])))

#### 4.2. Многочлен
Найдите корень (он один) заданного полинома (очень хорошего!) с точностью до пяти знаков после запятой:
1. Используя бинарный поиск https://en.wikipedia.org/wiki/Binary_search_algorithm
2. Используя метод Ньютона https://en.wikipedia.org/wiki/Newton%27s_method
   
   Задаётся начальное приближение вблизи предположительного корня, после чего строится касательная к графику исследуемой функции в точке приближения, для которой находится пересечение с осью абсцисс. Эта точка берётся в качестве следующего приближения. И так далее, пока не будет достигнута необходимая точность.
   
   (hint: для вычисления производных используйте метод `backward()`)
   
   $x_{n+1} = x_{n} - \frac{f(x_n)}{f'(x_n)}$

Сравните скорость методов с помощью `%%timeit`, т.е. оцените, какой из них найдёт ответ быстрее

In [ ]:
from typing import Callable
Polynomial = Callable[[torch.FloatTensor], torch.FloatTensor]

def poly(x: torch.FloatTensor) -> torch.FloatTensor:
    return x ** 7 + 5 * x ** 3 + 17 * x - 9

In [ ]:
def bin_search_find_zero(poly: Polynomial) -> torch.FloatTensor:
  """Функция для бинарного поиска"""
  ...
  return ...

In [ ]:
def newton_find_zero(poly: Polynomial) -> torch.FloatTensor:
    """Функция для метода Ньютона"""

    # первое приближение (не забываем про то, что понадобится градиент!)
    x = ...

    # останавливаемся, если значение функции достаточно близко к нулю
    tol = 10 ** -5

    # значение 
    val = ...

    # цикл обновления
    while ...:  # когда останавливаемся?
        # получаем градиент, обновляем значение x, оцениваем f(x)
        # hint: нужны ли нам градиенты, когда мы обновляем x?
        # hint: не забываем про обнуление градиента с прошлых шагов
        ...
    
    return x

In [ ]:
x = newton_find_zero(poly)
print(x)
print(poly(x))


In [ ]:
%%timeit
x = newton_find_zero(poly)

### 6. Что почитать / посмотреть
1. [Backpropagation: анимированное изложение](https://developers-dot-devsite-v2-prod.appspot.com/machine-learning/crash-course/backprop-scroll)
2. [You should understand backprop](https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b) от Andrej Karpathy
3. [Neural Networks: Zero to Hero](https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)